In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import copy
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_all = pd.read_csv('/content/drive/My Drive/datasets/data_all.csv')
dict_all = pd.read_csv('/content/drive/My Drive/datasets/dict_before_spacy.csv')
indeksY_pod = pd.read_csv('/content/drive/My Drive/datasets/indeksy_pod_idf.csv')

In [4]:
dict_all = dict_all.drop("Unnamed: 0", 1)
data_all = data_all.drop("Unnamed: 0", 1)

In [5]:
data_all

,docID,count,word
0,1,1,access
1,1,1,additional
2,1,1,april
3,1,1,authorize
4,1,1,basis
...,...,...,...
1390492,7500,1,transitory
1390493,7500,1,treated
1390494,7500,1,under
1390495,7500,1,year


In [6]:
data = [dict_all.reset_index()['index'], dict_all['word']]
headers = ["wordID", "word"]
df_temp = pd.concat(data, axis = 1, keys = headers)
essa = pd.merge(data_all, df_temp, how='left', on='word')

In [7]:
essa = essa.drop("word", 1)

In [8]:
essa

,docID,count,wordID
0,1,1,76
1,1,1,202
2,1,1,808
3,1,1,1108
4,1,1,1342
...,...,...,...
1390492,7500,1,33411
1390493,7500,1,16449
1390494,7500,1,49524
1390495,7500,1,24954


In [9]:
indeksY_pod

,Unnamed: 0,idf
0,18,4.379364
1,19,2.430418
2,20,2.083182
3,34,3.316856
4,38,3.425490
...,...,...
4442,48269,4.047461
4443,49029,4.933674
4444,49248,4.411799
4445,49393,5.010635


In [10]:
lista_indeksow = indeksY_pod['Unnamed: 0'].to_list()

In [12]:
len(lista_indeksow)

4447

In [14]:
essa_fixed = essa[essa['wordID'].isin(lista_indeksow)]

In [15]:
essa_fixed

,docID,count,wordID
0,1,1,76
1,1,1,202
2,1,1,808
4,1,1,1342
5,1,1,3447
...,...,...,...
1390487,7500,1,14426
1390491,7500,1,15981
1390493,7500,1,16449
1390494,7500,1,49524


In [16]:
document_lengths = essa_fixed.groupby(['docID', 'wordID'])['count'].count()

In [17]:
document_lengths

docID  wordID
1      76        1
       202       1
       808       1
       1342      1
       3447      1
                ..
7500   16449     1
       20563     1
       24954     1
       48269     1
       49524     1
Name: count, Length: 1038448, dtype: int64

In [18]:
document_freqs = essa_fixed.groupby('wordID')['docID'].count()
document_freqs = pd.DataFrame(document_freqs)
document_freqs.rename(columns={'docID': 'df'}, inplace=True)
document_freqs

,df
wordID,
18,94
19,660
20,934
34,272
38,244
...,...
48269,131
49029,54
49248,91


In [ ]:
sum(document_freqs['df'])

1390497

In [19]:
document_freqs.describe()

,df
count,4447.000000
mean,233.516528
std,238.970283
min,5.000000
25%,89.000000
50%,149.000000
75%,280.000000
max,2087.000000


In [20]:
dfs = np.array(document_freqs['df'])
dfs

array([ 94, 660, 934, ...,  91,  50, 138])

In [21]:
idfs = np.log(7500/dfs)
document_freqs['idf'] = idfs
document_freqs

,df,idf
wordID,,
18,94,4.379364
19,660,2.430418
20,934,2.083182
34,272,3.316856
38,244,3.425490
...,...,...
48269,131,4.047461
49029,54,4.933674
49248,91,4.411799


In [ ]:
essa_testyn = essa.drop("word", 1)

In [ ]:
essa

,docID,count,wordID
0,1,1,76
1,1,1,202
2,1,1,808
3,1,1,1108
4,1,1,1342
...,...,...,...
1390492,7500,1,33411
1390493,7500,1,16449
1390494,7500,1,49524
1390495,7500,1,24954


In [ ]:
testyn = essa.pivot('docID', 'wordID', 'count')

In [ ]:
testyn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7500 entries, 1 to 7500
Columns: 49760 entries, 0 to 49759
dtypes: float64(49760)
memory usage: 2.8 GB


In [ ]:
testyn = testyn.fillna(0)

In [ ]:
testyn

wordID,0,1,2,3,4,5,6,7,8,9,...,49750,49751,49752,49753,49754,49755,49756,49757,49758,49759
docID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
document_freqs.to_csv('idf_of_all_words_fixed.csv')

In [ ]:
testyn.to_csv('counters_all_words.csv')